In [1]:
import os
import datasets
import numpy as np
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
from knowledge_distillation import DistillationTrainingArguments, DistillationTrainer, TinyTrainer
from huggingface_hub import notebook_login
from dotenv import load_dotenv
load_dotenv(verbose=True)
import torch
import wandb
import sys
sys.path.append('/opt/ml/final-project-level3-nlp-02')
from rouge import compute

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
wandb.init(
    entity="final_project",
    project='optimization',
    name='edpl3_tiny_25ep'
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: abbymark (use `wandb login --relogin` to force relogin)


In [8]:
seed = 42

train_size = 5000
eval_size = 500
check_point = 'encoder_decoder_pruned_last_5' #"gogamza/kobart-summarization"
max_input_length = 512
max_target_length = 30

batch_size = 8
num_train_epochs = 25
learning_rate=5.6e-5
weight_decay = 0.01
logging_steps = 500
model_name = check_point.split("/")[-1]

distillation_type = 'tiny'
if distillation_type:
    student_check_point = "encoder_decoder_pruned_last_3"
    teacher_check_point = "kobart-summarization-finetuned-5000/checkpoint-1000"
    alpha=0.5
    temperature = 30


# Loading Dataset

In [9]:
api_token = os.getenv('HF_DATASET_API_TOKEN')
dataset = datasets.load_dataset('metamong1/summarization_paper', use_auth_token=api_token)

Reusing dataset paper_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/1.4.0/24bb09528ebb04fdc6aafb6e110202e52fbb818c0f204839bc833d8ce1e86a5f)


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
training_dataset = dataset['train'].shuffle(seed=seed).select(range(train_size))

Loading cached shuffled indices for dataset at /opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/1.4.0/24bb09528ebb04fdc6aafb6e110202e52fbb818c0f204839bc833d8ce1e86a5f/cache-93a7be41ce043bd9.arrow


# Prepare Training

In [11]:
if distillation_type:
    tokenizer = AutoTokenizer.from_pretrained(student_check_point)
else:
    tokenizer = AutoTokenizer.from_pretrained(check_point)

In [12]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples['text'], max_length=max_input_length, truncation = True, #padding=True
    )

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['title'], max_length=max_target_length, truncation=True, #padding=True
        )
    
    model_inputs['labels'] = labels['input_ids']

    return model_inputs

In [13]:
tokenized_train_dataset = training_dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/1.4.0/24bb09528ebb04fdc6aafb6e110202e52fbb818c0f204839bc833d8ce1e86a5f/cache-0083b6d5c7c91f98.arrow


In [14]:
tokenized_eval_dataset = dataset['validation'].select(range(500)).map(preprocess_function, batched=True)

Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/1.4.0/24bb09528ebb04fdc6aafb6e110202e52fbb818c0f204839bc833d8ce1e86a5f/cache-d9529bd184d76b26.arrow


# Training

In [15]:
if distillation_type:
    student_model = AutoModelForSeq2SeqLM.from_pretrained(student_check_point, torch_dtype='auto').to(device)
    teacher_model = AutoModelForSeq2SeqLM.from_pretrained(teacher_check_point, torch_dtype='auto').to(device)
else:
    model = AutoModelForSeq2SeqLM.from_pretrained(check_point, torch_dtype='auto').to(device)

/opt/conda/envs/final/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [16]:
if distillation_type:
    args =  DistillationTrainingArguments(
        output_dir=f'{student_check_point}-distilled-{train_size}',
        evaluation_strategy = 'steps',
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay = weight_decay,
        save_total_limit=2,
        num_train_epochs=num_train_epochs,
        predict_with_generate=True,
        logging_steps=logging_steps,
        alpha=alpha,
        temperature=temperature,
        report_to='wandb'
    )
else:
    args = Seq2SeqTrainingArguments(
        output_dir=f'{model_name}-finetuned-{train_size}',
        evaluation_strategy='steps', #'epoch',
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay = weight_decay,
        save_total_limit=2,
        num_train_epochs=num_train_epochs,
        predict_with_generate=True,
        logging_steps=logging_steps,
        report_to='wandb'
        # push_to_hub=True,
    )

In [17]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE scores
    result = compute(
        predictions=decoded_preds, references=decoded_labels, tokenizer=tokenizer
    )

    # Extract the median scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [18]:
if distillation_type:
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=student_model)
else:
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [19]:
if distillation_type == 'distil':
    trainer = DistillationTrainer(
        model=student_model,
        args=args,
        teacher_model = teacher_model,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_eval_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
elif distillation_type == 'tiny':
    trainer = TinyTrainer(
        model=student_model,
        args=args,
        teacher_model = teacher_model,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_eval_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
else:
    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_eval_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics, 
    )

In [20]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: doc_id, text, title, file, doc_type.
***** Running training *****
  Num examples = 5000
  Num Epochs = 25
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 15625
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
500,18.237900,6.121234,5.675700,0.556100,5.203200,5.184500
1000,12.976100,5.468546,7.258400,0.796900,6.486100,6.486900
1500,11.283900,4.995747,8.367400,1.126200,7.582600,7.540300
2000,10.315500,4.619568,10.087900,1.865300,8.902800,8.877800
2500,9.569400,4.279552,12.936100,3.434900,11.557800,11.542500
3000,8.836500,3.957933,15.409200,5.014300,13.835600,13.800700
3500,8.207200,3.671987,19.279800,7.896200,17.405700,17.412900
4000,7.847700,3.427720,21.823600,9.859300,19.892000,19.913200
4500,7.380900,3.231940,24.637900,11.965300,22.385900,22.339400
5000,7.007700,3.111772,26.233000,14.074700,24.101500,24.100900


The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: doc_id, text, title, file, doc_type.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to encoder_decoder_pruned_last_3-distilled-5000/checkpoint-500
Configuration saved in encoder_decoder_pruned_last_3-distilled-5000/checkpoint-500/config.json
Model weights saved in encoder_decoder_pruned_last_3-distilled-5000/checkpoint-500/pytorch_model.bin
tokenizer config file saved in encoder_decoder_pruned_last_3-distilled-5000/checkpoint-500/tokenizer_config.json
Special tokens file saved in encoder_decoder_pruned_last_3-distilled-5000/checkpoint-500/special_tokens_map.json
Deleting older checkpoint [encoder_decoder_pruned_last_3-distilled-5000/checkpoint-30500] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forwar

KeyboardInterrupt: 

In [57]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: title, file, doc_type, text, doc_id.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 3.3818862438201904,
 'eval_rouge1': 31.6557,
 'eval_rouge2': 18.708,
 'eval_rougeL': 28.8796,
 'eval_rougeLsum': 28.8763,
 'eval_runtime': 8.5031,
 'eval_samples_per_second': 58.802,
 'eval_steps_per_second': 7.409,
 'epoch': 10.0}